In [42]:
from Mapping.Cell import *
from Mapping.BoundingBox import BoundingBox
from Mapping.EnvironmentMap import EnvironmentMap
from random import random
import plotly.graph_objects as go
from Mapping.Octree import Octree
import numpy as np
import json

In [43]:
with open("./config.json", 'r') as f:
    config = json.load(f)

In [44]:
config_front = config['agents'][0]['sensors'][-2]["configuration"]
theta_front = np.linspace(-config_front["Azimuth"]/2, config_front["Azimuth"]/2, config_front["AzimuthBins"])*np.pi/180
r_front = np.linspace(config_front["RangeMin"], config_front['RangeMax'], config_front["RangeBins"])
T_front, R_front = np.meshgrid(theta_front, r_front)
z_front = np.zeros_like(T_front)

config_depth = config['agents'][0]['sensors'][-1]["configuration"]
theta_depth = np.linspace(-config_depth["Azimuth"]/2, config_depth["Azimuth"]/2, config_depth["AzimuthBins"])*np.pi/180
r_depth = np.linspace(config_depth["RangeMin"], config_depth['RangeMax'], config_depth["RangeBins"])
T_depth, R_depth = np.meshgrid(theta_depth, r_depth)
z_depth = np.zeros_like(T_depth)

In [46]:
front_ak = config_front['RangeMax'] / np.cos(np.deg2rad(config_front["Azimuth"]/2.))
boundary_relativ_front : BoundingBox = BoundingBox((config_front["RangeMin"], -front_ak, 0), (config_front['RangeMax'], front_ak, 0)) # x,y. z is fixed

z_tmp = np.cos(theta_depth)*config_depth['RangeMax']
z_rel_uncovered = [z_tmp[0], z_tmp[-1]]
z_rel_covered = z_tmp[1:-1]
del z_tmp

y_tmp = np.sin(theta_depth)*config_depth['RangeMax']
y_rel_covered = y_tmp[1:-1]
y_rel_uncovered = [y_tmp[0], y_tmp[-1]]
del y_tmp

x_rel_covered = np.sin(np.deg2rad(config_depth["Elevation"]))*config_depth['RangeMax']
depth_gk = z_rel_uncovered[0]
depth_ak = y_rel_uncovered[-1]
delta = 0.01
boundary_relativ_depth : BoundingBox = BoundingBox((-x_rel_covered, -depth_ak - delta, -config_depth['RangeMax'] - delta), (x_rel_covered, depth_ak + delta, -config_depth['RangeMin'] + delta))

numerator = 2000
correct_theta = True
correct_z = False

In [47]:
def scatter_plot(x,y,z):
    
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                    mode='markers',
                                    marker=dict(
                                        size=2,
                                        color=z,
                                        colorscale='blackbody'
                                    ))]
                )
    fig.show()

In [102]:
degree = 70

triangle_depth_side = [(0, hyp*np.sin(theta), -hyp*np.cos(theta) + 0) for hyp in np.arange(config_depth["RangeMin"],config_depth['RangeMax'], 0.5) for theta in [theta_depth[0], theta_depth[-1]]]
triangle_depth_bottom = [(0, y, -z_rel_uncovered[0] + 0) for y in np.append(y_rel_covered, y_rel_uncovered)]

unex_depth = triangle_depth_bottom + triangle_depth_side
unex_front = [(hyp*np.cos(theta),-hyp*np.sin(theta), 0) for hyp in np.arange(config_depth["RangeMin"], config_depth['RangeMax'], 0.5) for theta in theta_front[1:-1]]

unex_depth_rot = [(np.cos(np.radians(degree))*pos[1] - np.sin(np.radians(degree))*pos[0], np.sin(np.radians(degree))*pos[1] + np.cos(np.radians(degree))*pos[0] , pos[2]) for pos in unex_depth]
unex_front_rot = [(np.cos(np.radians(degree))*pos[1] - np.sin(np.radians(degree))*pos[0], np.sin(np.radians(degree))*pos[1] + np.cos(np.radians(degree))*pos[0] , pos[2]) for pos in unex_front]

all_unexplored_points = [(np.cos(np.radians(degree))*pos[1] - np.sin(np.radians(degree))*pos[0], np.sin(np.radians(degree))*pos[1] + np.cos(np.radians(degree))*pos[0] , pos[2]) for pos in unex_depth + unex_front]
x,y,z = zip(*all_unexplored_points)
scatter_plot(x,y,z)